In [1]:
import sys
print(sys.version)

3.8.10 (default, Mar 15 2022, 12:22:08) 
[GCC 9.4.0]


In [2]:
import pandas as pd

In [3]:
from wefe.query import Query
from wefe.metrics import WEAT, RNSB, RND, MAC, ECT
from wefe.word_embedding_model import WordEmbeddingModel
from wefe.datasets.datasets import load_weat, fetch_eds, fetch_debias_multiclass, fetch_debiaswe, load_bingliu
from wefe.utils import run_queries, plot_queries_results, create_ranking, plot_ranking, plot_ranking_correlations
from plotly.subplots import make_subplots
import gensim

In [4]:
data_folder = "../../Data/"
embeddings_folder = data_folder+"/word embeddings/"

In [5]:
def get_google_news_embeddings(filename,binary=False):
    w2v_model = gensim.models.KeyedVectors.load_word2vec_format(filename, binary=binary, no_header=True)
    return w2v_model


In [6]:
u_desip_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"debiased_word_embedding/glove_2_wv_U-DeSIP.txt", binary=False)

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/debiased_word_embedding/glove_2_wv_U-DeSIP.txt
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/debiased_word_embedding/glove_2_wv_U-DeSIP.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (322636, 300) matrix of type float32 from ../../Data//word embeddings/debiased_word_embedding/glove_2_wv_U-DeSIP.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T12:47:15.343012', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [7]:
sswe_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"/SSWE/embedding-results/sswe-u.bin",binary=True)

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings//SSWE/embedding-results/sswe-u.bin
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings//SSWE/embedding-results/sswe-u.bin', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (137052, 50) matrix of type float32 from ../../Data//word embeddings//SSWE/embedding-results/sswe-u.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2022-04-19T12:51:05.441221', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [8]:
p_desip_model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_folder+"debiased_word_embedding/glove_2_wv_P-DeSIP.txt", binary=False)

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings/debiased_word_embedding/glove_2_wv_P-DeSIP.txt
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings/debiased_word_embedding/glove_2_wv_P-DeSIP.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (322636, 300) matrix of type float32 from ../../Data//word embeddings/debiased_word_embedding/glove_2_wv_P-DeSIP.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T12:51:29.725620', 'gensim': '4.0.1', 'python': '3.8.10 (default, Mar 15 2022, 12:22:08) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-107-generic-x86_64-with-glibc2.29', 'event': 'load_word2vec_format'}


In [9]:
glove_cc_model = get_google_news_embeddings(embeddings_folder+"/Glove/glove.42B.300d.txt")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings//Glove/glove.42B.300d.txt
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings//Glove/glove.42B.300d.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings//Glove/glove.42B.300d.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (1917494, 300) matrix of type float32 from ../../Data//word embeddings//Glove/glove.42B.300d.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T12:53:47.653659', '

In [10]:
glove_cc_large_model = get_google_news_embeddings(embeddings_folder+"/Glove/glove.840B.300d.txt")

INFO:gensim.models.keyedvectors:loading projection weights from ../../Data//word embeddings//Glove/glove.840B.300d.txt
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings//Glove/glove.840B.300d.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:smart_open.smart_open_lib:{'uri': '../../Data//word embeddings//Glove/glove.840B.300d.txt', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'compression': None, 'transport_params': None}
DEBUG:gensim.utils:starting a new internal lifecycle event log for KeyedVectors
INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (2196017, 300) matrix of type float32 from ../../Data//word embeddings//Glove/glove.840B.300d.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T12:56:26.468187

In [11]:
# load the weat word sets
WEAT_wordsets = load_weat()
RND_wordsets = fetch_eds()
sentiments_wordsets = load_bingliu()
debias_multiclass_wordsets = fetch_debias_multiclass()

#### WEAT Effect Size: represents a normalized measure that quantifies how far apart the two distributions of association between targets and attributes are.

#### WEAT score:  Its objective is to quantify the strength of association of both pair of sets through a permutation test. The score being positive and higher than one indicates that word2vec exhibits a moderately strong relationship between men's names and careers and women's names and family.

In [12]:
profane_words = open(data_folder+"swear_words.txt").read().splitlines()

In [13]:
harmless_minority_words_clean = ["african", "african american", "asian", "black",
                                 "hispanic", "latin", "mexican",
                                 "indian", "middle eastern", "arab"]

harmless_majority_words_clean = ["white","caucasian",
                                 "european american","european", "norwegian",
                                 "canadian", "german", "australian", "english",
                                "french", "american", "swedish", "dutch"]


#### running multiple queries and multiple word embeddings

In [14]:
u_desip_model = WordEmbeddingModel(u_desip_model, 'u_desip')
p_desip_model = WordEmbeddingModel(p_desip_model, 'p_desip')
sswe_model = WordEmbeddingModel(sswe_model, 'sswe')
glove_cc_model = WordEmbeddingModel(glove_cc_model, 'glove_cc')
glove_cc_large_model = WordEmbeddingModel(glove_cc_large_model, 'glove_cc_large')

In [15]:
models = [u_desip_model, p_desip_model, sswe_model, glove_cc_model, glove_cc_large_model]

## Query ranking

In [16]:
#query1 = Query([harmless_minority_words_clean, harmless_majority_words_clean],
   #                    [WEAT_wordsets['unpleasant_5'], WEAT_wordsets['pleasant_5']])

query2 = Query([harmless_minority_words_clean, harmless_majority_words_clean],
                       [profane_words, sentiments_wordsets['positive_words']])

gender_queries = [query2]

In [17]:
# Run the queries WEAT
WEAT_gender_results = run_queries(
    WEAT,
    gender_queries,
    models,
    aggregate_results=True,
    return_only_aggregation=True,
    lost_vocabulary_threshold = 0.6,
    queries_set_name='SOS Queries')

# Run the queries with WEAT Effect Size

WEAT_EZ_gender_results = run_queries(WEAT,
                                     gender_queries,
                                     models,
                                     metric_params={
                                        
                                         'return_effect_size': True
                                     },
                                     aggregate_results=True,
                                     return_only_aggregation=True,
                                     lost_vocabulary_threshold = 0.6,
                                     queries_set_name='SOS Queries')


# Run the queries using RNSB
RNSB_gender_results = run_queries(RNSB,
                                gender_queries,
                                models,
                                aggregate_results=True,
                                return_only_aggregation=True,
                                    lost_vocabulary_threshold = 0.6,
                                queries_set_name='SOS Queries')




/home/fatma/.local/lib/python3.8/site-packages/wefe/metrics/RNSB.py:150: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  probabilities = np.array([
/home/fatma/.local/lib/python3.8/site-packages/wefe/metrics/RNSB.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  negative_probabilities = np.array(
/home/fatma/.local/lib/python3.8/site-packages/wefe/metrics/RNSB.py:150: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. I

In [18]:
# Run the queries using RND
RND_gender_results = run_queries(RND,
                                gender_queries,
                                models,
                                aggregate_results=True,
                                generate_subqueries=True,
                                lost_vocabulary_threshold = 0.6,
                                queries_set_name='SOS Queries')




In [19]:
# Run the queries using ECT
ECT_gender_results = run_queries(ECT,
                                gender_queries,
                                models,
                                aggregate_results=True,
                                generate_subqueries=True,
                                lost_vocabulary_threshold = 0.6,
                                queries_set_name='SOS Queries')

In [20]:
WEAT_gender_results

,WEAT: SOS Queries average of abs values score
model_name,
u_desip,0.076516
p_desip,0.063753
sswe,1.765366
glove_cc,0.628459
glove_cc_large,0.274132


In [21]:
RNSB_gender_results

,RNSB: SOS Queries average of abs values score
model_name,
u_desip,0.811371
p_desip,0.878332
sswe,0.230516
glove_cc,1.493269
glove_cc_large,1.332916


In [22]:
RND_gender_results["RND: SOS Queries average of abs values score"]

model_name
u_desip           0.011194
p_desip           0.027578
sswe              0.306853
glove_cc          0.126138
glove_cc_large    0.206265
Name: RND: SOS Queries average of abs values score, dtype: float64

In [23]:
ECT_gender_results["ECT: SOS Queries average of abs values score"]

model_name
u_desip           0.816771
p_desip           0.804652
sswe              0.869437
glove_cc          0.966801
glove_cc_large    0.934579
Name: ECT: SOS Queries average of abs values score, dtype: float64

In [24]:
import numpy as np
import scipy

In [25]:
ECT_gender_results["ECT: SOS Queries average of abs values score"]

model_name
u_desip           0.816771
p_desip           0.804652
sswe              0.869437
glove_cc          0.966801
glove_cc_large    0.934579
Name: ECT: SOS Queries average of abs values score, dtype: float64

In [26]:
ECT_gender_results

,Target set 0 and Target set 1 wrt Attribute set 0,Target set 0 and Target set 1 wrt Attribute set 1,ECT: SOS Queries average of abs values score
model_name,,,
u_desip,0.770581,0.862961,0.816771
p_desip,0.755079,0.854225,0.804652
sswe,0.866476,0.872398,0.869437
glove_cc,0.958292,0.975310,0.966801
glove_cc_large,0.941969,0.927190,0.934579


In [27]:
gender_ranking = create_ranking([ WEAT_gender_results, 
                                RNSB_gender_results, RND_gender_results
                                ], ascending=True)
ect_gender_Ranking = create_ranking([ECT_gender_results], ascending=False)

In [28]:
ect_gender_Ranking

,ECT: SOS Queries average of abs values score
model_name,
u_desip,4.0
p_desip,5.0
sswe,3.0
glove_cc,1.0
glove_cc_large,2.0


In [29]:
gender_ranking["ECT"] = ect_gender_Ranking["ECT: SOS Queries average of abs values score"].values

In [30]:
gender_ranking

,WEAT: SOS Queries average of abs values score,RNSB: SOS Queries average of abs values score,RND: SOS Queries average of abs values score,ECT
model_name,,,,
u_desip,2.0,2.0,1.0,4.0
p_desip,1.0,3.0,2.0,5.0
sswe,5.0,1.0,5.0,3.0
glove_cc,4.0,5.0,3.0,1.0
glove_cc_large,3.0,4.0,4.0,2.0
